In [8]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import random
import sys

In [9]:
sys.path.insert(1, '/Users/manjugupta/Desktop/CMU_Courses/Intents/getting_intents/Analysis')

In [10]:
from get_vocab import load_data

In [11]:
#Check if cuda is available
cuda = torch.cuda.is_available()
print('CUDA is', cuda)

num_workers = 8 if cuda else 0

print(num_workers)

CUDA is False
0


In [12]:
language = 'english'
context_size = 2
emb_size = 22 # size of embeddings you want to create
embedding_data = load_data('embedding_data_' + language + '_' + str(context_size) + '.pkl')
vocab_size = len(embedding_data) + 1

print(embedding_data.keys())
print(vocab_size)

dict_keys(['h', 'x', 'ɹ', 'ʒ', 'aː', 'r', 'k', 'uː', 'æ', 'b', 'θ', 'a', 'eː', 'p', 'ɔ', 'e', 'ə', 'i', 'iː', 'm', 'f', 'z', 'ʊ', 'ɒ', 'ɛ', 'd', 'w', 'ŋ', 'ʌ', 'ʃ', 'ɑ', 'j', 'ð', 'ɪ', 's', 'ʔ', 'o', 'l', 'u', 'pʰ', 'ɯ', 'n', 'v', 'ɻ', 'tʰ', 'kʰ', 't', 'ɡ'])
49


In [13]:
feature_vectors = {}        #create a dictionary of feature vectors
for i, phone in enumerate(embedding_data):
    feature_vectors[phone] = [0]*(len(embedding_data) + 1)
    feature_vectors[phone][i] = 1
    
feature_vectors['unk'] = [0]*(len(embedding_data) + 1)
feature_vectors['unk'][len(embedding_data)] = 1

In [14]:
class MyDataset(Dataset):
    def __init__(self, embedding_data, feature_vectors):

        self.data = []
        for phone in embedding_data:
            for context in embedding_data[phone]:
                phone_to_vector = []
                for c in context:
                    phone_to_vector.append(feature_vectors[c])
                    
                label = np.argmax(np.array(feature_vectors[phone]))
                self.data.append([torch.from_numpy(np.array(phone_to_vector)).float(), label])
                
        random.shuffle(self.data)
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        
        return self.data[index][0], self.data[index][1]

In [15]:
train_dataset = MyDataset(embedding_data, feature_vectors)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=32)
train_loader = DataLoader(train_dataset, **train_loader_args)

In [16]:
len(train_loader)

5035

In [20]:
for data in train_loader:
    print(len(data))
    print(data[0].shape, data[1].shape)
    print(data[1])
    break

2
torch.Size([32, 4, 49]) torch.Size([32])
tensor([17, 34, 41, 41,  6, 46, 41, 15, 24, 30, 33, 11, 33, 24, 27, 31, 19, 19,
        33, 11, 34, 36,  8, 25, 34, 37, 33, 14, 36, 14, 16, 33])


In [21]:
class My_Model(nn.Module):
    def __init__(self, emb_size, vocab_size):
        super(My_Model, self).__init__()
        self.fc1 = nn.Linear(vocab_size, emb_size, bias = False)
        self.fc2 = nn.Linear(emb_size, vocab_size, bias = False)

    def forward(self, x):
        outputs = self.fc1(x[:,0,:])
        for i in range(1, x.shape[1]):
            outputs += self.fc1(x[:,i,:])
                           
        outputs /= outputs/x.shape[1]
                           
        output = self.fc2(outputs)   
        return output

In [27]:
model = My_Model(emb_size, vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)
print(optimizer)

My_Model(
  (fc1): Linear(in_features=49, out_features=22, bias=False)
  (fc2): Linear(in_features=22, out_features=49, bias=False)
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)


In [28]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device

        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += target.size(0)
        correct_predictions += (predicted == target).sum().item()
    
            
    end_time = time.time()
    
    acc = (correct_predictions/total_predictions)*100.0
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')  
    print('Training Accuracy: ', acc, '%')
    return running_loss

In [30]:
Train_loss = []
Test_loss = []
Test_acc = []
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, cooldown=5)

for i in range(100):
    print('epoch:', i)
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    Train_loss.append(train_loss)

epoch: 0
Training Loss:  3.598143456306571 Time:  7.013299942016602 s
Training Accuracy:  6.731598909932895 %
epoch: 1
Training Loss:  3.5978163810093564 Time:  6.887549161911011 s
Training Accuracy:  6.894239901670485 %
epoch: 2
Training Loss:  3.5977098687521396 Time:  7.884326934814453 s
Training Accuracy:  6.875616887349387 %
epoch: 3
Training Loss:  3.5976403592004558 Time:  7.110896110534668 s
Training Accuracy:  6.799883295776921 %
epoch: 4
Training Loss:  3.601850768991828 Time:  8.692988157272339 s
Training Accuracy:  6.7198043341961995 %
epoch: 5
Training Loss:  3.603662505665981 Time:  6.869415998458862 s
Training Accuracy:  6.737806581373261 %
epoch: 6
Training Loss:  3.597650813346113 Time:  7.665459156036377 s
Training Accuracy:  6.794917158624628 %
epoch: 7
Training Loss:  3.5992199381153065 Time:  8.610066890716553 s
Training Accuracy:  6.8731338187732405 %
epoch: 8
Training Loss:  3.5985020568851445 Time:  8.213409185409546 s
Training Accuracy:  6.828438584402605 %
epo

KeyboardInterrupt: 

In [138]:
model.fc1.weight.shape

torch.Size([22, 49])